In [6]:
import pandas as pd
import numpy as np
import datetime as dt
import openpyxl as xls

In [7]:
prices=pd.read_csv(r'C:\data analyst project data\airbnb_price.csv')
prices.head()

,listing_id,price,nbhood_full
0,2595,225 dollars,"Manhattan, Midtown"
1,3831,89 dollars,"Brooklyn, Clinton Hill"
2,5099,200 dollars,"Manhattan, Murray Hill"
3,5178,79 dollars,"Manhattan, Hell's Kitchen"
4,5238,150 dollars,"Manhattan, Chinatown"


In [8]:
room_type=pd.read_excel(r'C:\data analyst project data\airbnb_room_type (2).xlsx')

In [9]:
reviews=pd.read_csv(r'C:\data analyst project data\airbnb_last_review.tsv',sep="\t")

In [10]:
print(prices.head(),"\n",room_type.head(),"\n",reviews.head())

   listing_id        price                nbhood_full
0        2595  225 dollars         Manhattan, Midtown
1        3831   89 dollars     Brooklyn, Clinton Hill
2        5099  200 dollars     Manhattan, Murray Hill
3        5178   79 dollars  Manhattan, Hell's Kitchen
4        5238  150 dollars       Manhattan, Chinatown 
    listing_id                                description        room_type
0        2595                      Skylit Midtown Castle  Entire home/apt
1        3831            Cozy Entire Floor of Brownstone  Entire home/apt
2        5099  Large Cozy 1 BR Apartment In Midtown East  Entire home/apt
3        5178            Large Furnished Room Near B'way     private room
4        5238         Cute & Cozy Lower East Side 1 bdrm  Entire home/apt 
    listing_id    host_name   last_review
0        2595     Jennifer   May 21 2019
1        3831  LisaRoxanne  July 05 2019
2        5099        Chris  June 22 2019
3        5178     Shunichi  June 24 2019
4        5238          

# Cleaning the price column

In [11]:
prices["price"]=prices["price"].str.replace("dollars","")


In [12]:
prices["price"]=pd.to_numeric(prices["price"])

In [13]:
print(prices["price"].describe())

count    25209.000000
mean       141.777936
std        147.349137
min          0.000000
25%         69.000000
50%        105.000000
75%        175.000000
max       7500.000000
Name: price, dtype: float64


# Calculating average price

In [14]:
free_listing=prices["price"]==0


In [15]:
prices=prices.loc[~free_listing]

In [16]:
avg_price=round(prices["price"].mean(),2)

In [17]:
print("The average price per night for an Airbnb listing in NYC ${}.".format(avg_price))

The average price per night for an Airbnb listing in NYC $141.82.


# Comparing costs to the private rental market

In [18]:
prices["price_per_month"]=prices["price"]*365/12

In [19]:
avg_price_per_month=round(prices["price_per_month"].mean(),2)

In [20]:
print("the average price per month for an Airbnb listing in NYC ${} ,while in private market you would pay {}".format(avg_price_per_month,"$3,100"))

the average price per month for an Airbnb listing in NYC $4313.61 ,while in private market you would pay $3,100


# cleaning room type

In [21]:

room_type["room_type"]=room_type["room_type"].str.lower()

In [22]:
# Update the room_type column to category data type
room_type["room_type"]=room_type["room_type"].astype("category")

In [23]:
room_frequencies=room_type["room_type"].value_counts()
print(room_frequencies)

entire home/apt    13266
private room       11356
shared room          587
Name: room_type, dtype: int64


# viewing reviews

In [24]:
#changing the datatype of the last review column to datetime
reviews["last_review"]=pd.to_datetime(reviews["last_review"])

In [25]:
first_reviewed=reviews["last_review"].dt.date.min()

In [26]:
last_reviewed=reviews["last_review"].dt.date.max()

In [27]:
# Print the oldest and newest reviews from the DataFrame
print("the earliest review of Airbnb is {},the latest review is {}.".format(first_reviewed,last_reviewed))

the earliest review of Airbnb is 2019-01-01,the latest review is 2019-07-09.


# joining dataframe

In [33]:
# Merge prices and room_types to create rooms_and_prices
rooms_prices=prices.merge(room_type,how='outer',on='listing_id')

# Merge rooms_and_prices with the reviews DataFrame to create airbnb_merged
airbnb_merged=rooms_prices.merge(reviews,how="outer",on='listing_id')

# Drop missing values from airbnb_merged
airbnb_merged.dropna(inplace=True)

print("there is {} duplicates in data".format(airbnb_merged.duplicated().sum()))


there is 0 duplicates in data


In [34]:
airbnb_merged.head()

,listing_id,price,nbhood_full,price_per_month,description,room_type,host_name,last_review
0,2595,225.0,"Manhattan, Midtown",6843.750000,Skylit Midtown Castle,entire home/apt,Jennifer,2019-05-21
1,3831,89.0,"Brooklyn, Clinton Hill",2707.083333,Cozy Entire Floor of Brownstone,entire home/apt,LisaRoxanne,2019-07-05
2,5099,200.0,"Manhattan, Murray Hill",6083.333333,Large Cozy 1 BR Apartment In Midtown East,entire home/apt,Chris,2019-06-22
3,5178,79.0,"Manhattan, Hell's Kitchen",2402.916667,Large Furnished Room Near B'way,private room,Shunichi,2019-06-24
4,5238,150.0,"Manhattan, Chinatown",4562.500000,Cute & Cozy Lower East Side 1 bdrm,entire home/apt,Ben,2019-06-09


# Analysing listing prices by NYC borough


In [38]:

# Extract information from the nbhood_full column and store as a new column, borough
airbnb_merged["borough"]=airbnb_merged["nbhood_full"].str.partition(',')[0]

# Group by borough and calculate summary statistics
boroughs=airbnb_merged.groupby("borough")["price"].agg(["sum","mean","count","median"])

# Round boroughs to 2 decimal places, and sort by mean in descending order
boroughs=boroughs.round(2).sort_values('mean',ascending=False)

print(boroughs)

                     sum    mean  count  median
borough                                        
Manhattan      1898417.0  184.04  10315   149.0
Brooklyn       1275250.0  122.02  10451    95.0
Queens          320715.0   92.83   3455    70.0
Staten Island    22974.0   86.04    267    71.0
Bronx            55156.0   79.25    696    65.0


# Price range by borough

In [48]:
# Create labels for the price range, label_names
lable_names=['Budget','Average','Expensive','Extravagent']

# Create the label ranges, ranges
ranges=[0,69,175,350,np.inf]

# Insert new column, price_range, into DataFrame
airbnb_merged["price_range"]=pd.cut(airbnb_merged["price"],bins=ranges,labels=lable_names)

# Calculate borough and price_range frequencies, prices_by_borough
prices_by_borough=airbnb_merged.groupby(["borough","price_range"])["price_range"].count()
print(prices_by_borough)

borough        price_range
Bronx          Budget          381
               Average         285
               Expensive        25
               Extravagent       5
Brooklyn       Budget         3194
               Average        5532
               Expensive      1466
               Extravagent     259
Manhattan      Budget         1148
               Average        5285
               Expensive      3072
               Extravagent     810
Queens         Budget         1631
               Average        1505
               Expensive       291
               Extravagent      28
Staten Island  Budget          124
               Average         123
               Expensive        20
               Extravagent       0
Name: price_range, dtype: int64
